<a href="https://colab.research.google.com/github/HBocaccio/NumLabIMC/blob/main/Colabs/nb10_DiferenciasFinitas_PDEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introducción al modelado continuo** (*a.k.a.* **Ecuaciones de la física matemática**)
## Laboratorio numérico 2024

Bienvenidos al laboratorio numérico de la materia *Introducción al modelado continuo*, también conocida como *Ecuaciones de la física matemática*. En este, vamos a ver métodos numéricos para resolver distintos tipos de problemas de manera general, y su relación y aplicación a problemas específicos de la materia, de manera tal de complementar los contenidos de la cursada teórico-práctica.

# Motivación de este colab

En el 1er eje temático de la materia vimos ecuaciones diferenciales ordinarias (ODEs), que describían la dinámica de variables continuas que dependían sólo del tiempo, es decir, cada una de las variables podía representarse con una función univariada (depende de una única variable). De esta manera, las ODEs que describían un sistema autónomo (el tiempo no aparece explícitamente en la ecuación) se escribían como $\dot{\vec{x}}(t) = \vec{f}(\vec{x})$. En el caso más simple, el 1D, teníamos una única variable descrita por una única ODE:

$$
\dot{x}(t) = f(x)
$$

Este sistema los podíamos resolver integrando numéricamente de manera tal de obtener la solución $x(t)$ en el marco de los problemas de valores iniciales.

En este eje temático vamos a abordar un problema distinto, que consiste en tener variables continuas que se representan como funciones multivariadas $u(x,t)$. De esta manera, la derivada respecto de una de sus variables ya no es la derivada total, si no que es una derivada parcial. Ahora las reglas que describen la evolución temporal de estas variables ya no son ODEs, si no que son **ecuaciones en derivadas parciales** (PDEs), que son básicamente ecuaciones que relacionan funciones multivariadas y sus derivadas parciales.

Este tema es de gran relevancia debido a que las PDEs son fundamentales en muchas áreas de la física y las matemáticas aplicadas, describiendo muchísimos fenómenos diversos como la difusión del calor, las ondas y la mecánica de fluidos. La resolución de estos sistemas se vuelve altamente no trivial, debiendo recurrir a distintas estrategias matemáticas (como las vistas en la teórica) y numéricas.

En los próximos colabs vamos a ver la implementación de algunos métodos numéricos. En particular, en este colab vamos a ver el método de diferencias finitas, que permite resolver estas ecuaciones, siendo especialmente útil cuando las soluciones analíticas no son posibles.

## Ecuaciones en Derivadas Parciales (PDEs)

Las PDEs involucran derivadas de una función con respecto a más de una variable independiente, a diferencia de las ODEs que tenían una única variable independiente. Entonces vamos a tener un campo escalar $u(x,t)$, que puede depender por ejemplo del tiempo y de una variable espacial. Para describir las derivadas parciales de $u(x,t)$ muchas veces se usa la siguiente notación $\frac{\partial u(x,t)}{\partial t} = u_t$.

Algunos ejemplos típicos de PDEs son:

- **Ecuación de difusión/calor:**
$$
u_t = \alpha u_{xx}
$$
- **Ecuación de ondas:**
$$
u_{tt} = c^2 u_{xx}
$$
- **Ecuación de Laplace:**
$$
\nabla^2 u = 0
$$
- **Ecuación de Poisson:**
$$
\nabla^2 u = f(x, y)
$$

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

---
# Método de Diferencias Finitas

El método de diferencias finitas es un método que ya vimos en el Colab de la clase de derivación numérica. En esa clase, básicamente aproximábamos las derivadas primeras por el cociente incremental, para obtener la derivada hacia adelante. Pero vimos que esto podía pensarse de manera generalizada a partir del desarrollo de Taylor de la función, para obtener las [fórmulas de diferencias finitas](https://en.wikipedia.org/wiki/Finite_difference#Forward,_backward,_and_central_differences).

Para una función $u(x)$, la derivada primera se puede aproximar usando las siguientes fórmulas de diferencias finitas:

- **Derivada primera hacia adelante o progresiva (forward):**
$$
u_x \approx \frac{u(x+\Delta x) - u(x)}{\Delta x}
$$
- **Derivada primera hacia atrás o regresiva (backward):**
$$
u_x \approx \frac{u(x) - u(x-\Delta x)}{\Delta x}
$$
- **Derivada primera central:**
$$
u_x \approx \frac{u(x+\Delta x) - u(x-\Delta x)}{2\Delta x}
$$

Y recordemos que otra manera de pensarlo era con el desarrollo de Taylor truncado de la función $u(x)$, de donde obteníamos que el error de truncamiento era del orden de $\Delta x$ para las fórmulas forward y backward, y de $\Delta x^2$ para la fórmula central.



## Derivadas segundas

La estimación de derivadas segundas es crucial para la resolución numérica de ecuaciones en derivadas parciales (PDEs) que involucran este tipo de términos.

De manera análoga a lo que hicimos para la derivada primera, podemos aproximar la derivada segunda con el cociente incremental, pero ahora de las derivadas primeras:

- **Derivada segunda hacia adelante o progresiva (forward):**
$$
u_{xx}(x) \approx \frac{u(x+2\Delta x) - 2u(x+\Delta x) + u(x)}{(\Delta x)^2}
$$

- **Derivada segunda hacia atrás o regresiva (backward):**
$$
u_{xx}(x) \approx \frac{u(x) - 2u(x-\Delta x) + u(x-2\Delta x)}{(\Delta x)^2}
$$
- **Derivada segunda central:**
$$
u_{xx}(x) \approx \frac{u(x+\Delta x) - 2u(x) + u(x-\Delta x)}{(\Delta x)^2}
$$

Estas fórmulas son de orden $O((\Delta x)^2)$, lo que significa que el error decrece cuadráticamente con el tamaño del paso de tiempo $\Delta x$.

La fórmula más común y sencilla para aproximar la segunda derivada temporal es mediante diferencias finitas centrales.



### Ejercicio 1

Sea la función $u(x)=sin(x)$, cuál es su derivada segunda? Comparar lo esperado analíticamente con el resultado de calcularla con diferencias finitas centrales.

In [ ]:
# # # COMPLETAR

## Matrices de Diferenciación

Supongamos que tenemos un conjunto de valores discretos de una variable $u$. Podemos entonces usar el método de diferencias finitas para aproximar las derivadas utilizando diferencias entre los valores de la variable en puntos discretos de una malla. Este tipo de estrategia se aplica comúnmente para resolver PDEs en dominios discretizados.

Para implementar la derivada en una grilla de puntos, construimos **matrices de diferenciación**. Estas matrices aplican el operador de derivación a cualquier función $u$ discretizada en una grilla.
Para construir las matrices de diferenciación en el método de diferencias finitas, tomemos una grilla uniforme de $N$ puntos en el intervalo $[a, b]$, donde el espaciamiento entre puntos es $h = \frac{b - a}{N - 1}$. Supongamos que tenemos una variable $u(x)$ evaluada en los puntos $x_1, x_2, \dots, x_N$, y queremos aproximar las derivadas de $u$ en esos puntos. Las matrices de diferenciación actuarán sobre el vector $\mathbf{u} = [u(x_1), u(x_2), \dots, u(x_N)]^\top$.

Por un tema de simplicidad, podemos plantear la siguiente notación $u(x_i)=u_i$. Entonces, para el punto $i$, la derivada de $u$, usando por ejemplo la fórmula forward, sería:

$$
u'_i \approx \frac{u_{i+1} - u_i}{h}
$$

Esto quiere decir que para obtener el vector de derivadas $\mathbf{u'} = [u_1, u_2, \dots, u_N]$ podemos pensar en un problema de álgebra de matrices, tal que

$$
\mathbf{u'} = D \mathbf{u}^\top
$$

Donde la matriz D es una matriz rala en la que sólo sobreviven los elementos que me relacionan cada $u'_i$ con combinaciones de $u_{i-1}$, $u_{i}$, $u_{i+1}$. Esto sirve para el cálculo de las derivadas con cualquiera de las 3 fórmulas de diferencias finitas.

Entonces, para todos los puntos internos $i = 1, 2, \dots, N-1$, la matriz de diferencias finitas de primera derivada hacia adelante, $D_1$, toma la forma:

$$
D = \frac{1}{h}
\begin{bmatrix}
-1 & 1 & 0 & 0 & \dots & 0 \\
0 & -1 & 1 & 0 & \dots & 0 \\
0 & 0 & -1 & 1 & \dots & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & \dots & -1 & 1 \\
0 & 0 & 0 & 0 & \dots & 0 & 0
\end{bmatrix}
$$

donde la última fila queda usualmente definida por condiciones de contorno.

De la misma manera, podemos plantear que la matriz de diferencias finitas de la primera derivada central es:

$$
D = \frac{1}{2h}
\begin{bmatrix}
0 & 1 & 0 & 0 & \dots & 0 & 0 \\
-1 & 0 & 1 & 0 & \dots & 0 & 0 \\
0 & -1 & 0 & 1 & \dots & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \dots & -1 & 0 & 1 \\
0 & 0 & 0 & 0 & \dots & 0 & -1 & 0
\end{bmatrix}
$$

Para la **segunda derivada** $u''(x)$, podemos usar una aproximación centrada que involucra los puntos $x_{i-1}$, $x_i$, y $x_{i+1}$:

$$
u''_i \approx \frac{u_{i+1} - 2u_i + u_{i-1}}{h^2}
$$

Entonces, la matriz de diferencias finitas para la segunda derivada es:

$$
D = \frac{1}{h^2}
\begin{bmatrix}
-2 & 1 & 0 & 0 & \dots & 0 & 0 \\
1 & -2 & 1 & 0 & \dots & 0 & 0 \\
0 & 1 & -2 & 1 & \dots & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \dots & 1 & -2 & 1 \\
0 & 0 & 0 & 0 & \dots & 0 & 1 & -2
\end{bmatrix}
$$


### Condiciones de contorno

Las **condiciones de contorno** especifican el comportamiento de la solución en los bordes del dominio (por ejemplo, en los puntos $x_1$ y $x_N$ de una grilla de $N$ puntos). Estas condiciones son esenciales para completar el sistema de ecuaciones diferenciales y afectan directamente las primeras y últimas filas de las matrices de diferenciación. Existen varios tipos comunes de condiciones de contorno:

#### 1. Condiciones de Dirichlet

Las **condiciones de Dirichlet** fijan el valor de la función en los extremos del dominio. Por ejemplo, si queremos que $u(x_1) = \alpha$ y $u(x_N) = \beta$, simplemente se imponen estos valores en las posiciones correspondientes de $\mathbf{u}$:
$$
u_1 = \alpha, \quad u_N = \beta
$$

Para implementar estas condiciones en la matriz, reemplazamos la primera y última ecuación en el sistema por $u_1 = \alpha$ y $u_N = \beta$, eliminando la necesidad de calcular derivadas en esos puntos.

#### 2. Condiciones de Neumann

Las **condiciones de Neumann** especifican el valor de la derivada en los extremos. Por ejemplo, si $u'(x_1) = \gamma$ y $u'(x_N) = \delta$, la derivada en los extremos se puede aproximar usando una fórmula de diferencias finitas.

Para $u'(x_1) = \gamma$, se puede usar una diferencia hacia adelante:

$$
u'_1 \approx \frac{u_2 - u_1}{h} = \gamma \Rightarrow u_2 = u_1 + h \gamma
$$

Para $u'(x_N) = \delta$, se usa una diferencia hacia atrás:

$$
u'_N \approx \frac{u_N - u_{N-1}}{h} = \delta \Rightarrow u_{N-1} = u_N - h \delta
$$

Esto afecta la matriz de diferenciación al cambiar las primeras y últimas filas para reflejar estas derivadas en lugar de las derivadas internas.

Cuando tengo condiciones de borde periódicas, la relación es $u(a) = u(b)$. Esto significa que el último punto de la grilla se conecta al primero, permitiéndonos calcular derivadas en todos los nodos sin excepciones. Para implementar esta condición hacemos lo mismo que antes, ajustamos los valores de la primera y última fila en la matriz de diferenciación.


### Ejercicio 2

Implementar funciones que calculen matrices de diferenciación para calcular la derivada primera con los 3 métodos de diferencias finitas.
- Sobre un subconjunto adecuado de los nodos de una grilla de un cierto intervalo $[a,b]$,
-Sobre todos los nodos de la grilla, asumiendo condiciones de borde periódicas $u(a)=u(b)$.

Para esto, explore la construcción de matrices ralas con elementos diagonales, o cerca de la diagonal con funciones de Numpy.

Pruebe estas funciones por ejemplo para obtener la derivada primera de la función $f(x)=sin(x)$.

In [ ]:
# # # COMPLETAR

**Ejemplo**

Sea una función $u(x)$, resolvamos la siguiente ecuación diferencial con diferencias finitas

$$
u'' = x^2 - 2 \\
$$

Para condiciones de contorno $u(0) = 0$ y $u(3) = 2$


In [ ]:
def u_pp(x):
    return x**2 - 2

x1 = 0
x2 = 3
u1 = 0
u2 = 2

N = 100
x = np.linspace(x1, x2, N+1)
dx = x[1] - x[0]

M = np.diag(-2*np.ones(N+1), 0) + np.diag(np.ones(N), 1) + np.diag(np.ones(N), -1)
M[0, 0] = 1
M[0, 1] = 0
M[-1, -1] = 1
M[-1, -2] = 0

b = (u_pp(x))*dx**2
b[0] = u1
b[-1] = u2

M_inv = np.linalg.inv(M)
f = np.matmul(M_inv, b)
# f = M_inv @ b
# f = np.linalg.solve(M, b)
plt.plot(x, f)
plt.show()

### Ejercicio 3

Considere ahora la ecuación para una función $u(x)$, con sus condiciones de contorno

$$
u'' = 2 \\
u(0) = 0 \\
u(1) = 0 \\
$$

Piense a priori si se le ocurre cuál podría ser una solución, si tuviera que resolver analíticamente, pero sin hacerlo. Resuelva ahora con el método de diferencias finitas como el usado en el ejemplo anterior. [Hint: puede tener problemas con la dimensión de la función u_pp evaluada en x si no la define correctamente. Asegúrese que la salida de esta función tenga las mismas dimensiones que x, generando el vector de salida para que las tenga]


In [ ]:
# # # COMPLETAR

## Métodos explícitos e implícitos

El **método de diferencias finitas** es una técnica para aproximar derivadas en una grilla de puntos discretos mediante diferencias entre valores de la función en estos puntos. Este método se aplica tanto en el espacio (para derivadas espaciales) como en el tiempo (para derivadas temporales) en una ecuación diferencial.

Estos esquemas de diferencias finitas son muy comunes en la aproximación de derivadas para resolver PDEs, pero **no especifican si el método es explícito o implícito**.

### Método Explícito e Implícito

Los términos **explícito** e **implícito** describen cómo se avanza en el tiempo en la solución numérica de una PDE, **independientemente del método usado para aproximar derivadas**. Así que, una vez que hemos aproximado las derivadas usando diferencias finitas, podemos elegir entre un esquema explícito o implícito para resolver la ecuación en el tiempo.

- **Método Explícito**: Calcula los valores futuros de la solución en función de los valores actuales, lo cual permite resolver directamente la ecuación en cada paso de tiempo sin resolver un sistema de ecuaciones.

- **Método Implícito**: Calcula los valores futuros en función de sí mismos, lo cual lleva a resolver un sistema de ecuaciones en cada paso de tiempo para encontrar los valores en el siguiente paso temporal.

### Diferencias Finitas en Métodos Explícitos e Implícitos

Para ilustrar cómo el método de diferencias finitas se combina con los métodos explícitos o implícitos, consideremos la **ecuación de difusión**:

$$
\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}
$$

1. **Método de Diferencias Finitas en Esquema Explícito**: Si aplicamos diferencias finitas en el tiempo y en el espacio, podemos escribir:
  $$
  \frac{u(x, t + \Delta t) - u(x, t)}{\Delta t} = \alpha \frac{u(x + \Delta x, t) - 2u(x, t) + u(x - \Delta x, t)}{(\Delta x)^2}
  $$

  Resolviendo para $u(x, t + \Delta t)$, obtenemos una expresión en función de los valores actuales, lo cual constituye un esquema explícito.

2. **Método de Diferencias Finitas en Esquema Implícito**: Si, en cambio, aplicamos diferencias finitas de modo que los valores en el lado derecho de la ecuación están evaluados en $t + \Delta t$, obtenemos un sistema de ecuaciones para $u(x, t + \Delta t)$, lo cual requiere resolver un sistema de ecuaciones para cada paso de tiempo.


#### Diferencia entre Métodos Explícitos e Implícitos

Los **Métodos Explícitos** son simples de implementar, pero su estabilidad depende del tamaño de los pasos de tiempo y espacio. En problemas de difusión, por ejemplo, los métodos explícitos requieren que $\Delta t$ sea muy pequeño para asegurar la estabilidad.

Los **Métodos Implícitos** son más estables y permiten mayores pasos de tiempo sin comprometer la precisión, pero requieren resolver un sistema de ecuaciones en cada paso de tiempo, lo cual puede ser computacionalmente costoso.

En la aplicación a PDEs, cada método tiene sus limitaciones y se ajusta mejor a ciertas ecuaciones o tipos de problemas:

- **Ecuaciones de Difusión** (como la del calor): Se pueden resolver tanto con métodos explícitos (aunque requieren pasos de tiempo pequeños para ser estables) como con métodos implícitos.

- **Ecuaciones de Onda**: En este caso, los métodos explícitos de diferencias finitas funcionan bien porque las soluciones suelen ser estables para pasos de tiempo razonables, aunque también se pueden usar métodos implícitos.

- **Ecuación de Poisson o Laplace**: Estas son ecuaciones elípticas, por lo que los métodos implícitos son necesarios. Inclusive, el método de diferencias finitas funciona mejor para geometrías simples. Para otros casos es ya directamente recomendable explorar otras estrategias.


---
# Ecuación de Difusión/Calor

Consideremos la ecuación de difusión unidimensional:
$$
u_t = \alpha u_{xx}
$$

Se puede discretizar en una malla de puntos en el espacio $x$ y en el tiempo $t$. Utilizando una malla con espaciamiento $\Delta x$ y $\Delta t$, las aproximaciones de diferencias finitas nos llevan a la siguiente formulación:

$$
\frac{u_i^{n+1} - u_i^n}{\Delta t} = \alpha \frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{(\Delta x)^2}
$$

Despejando $u_i^{n+1}$:
$$
u_i^{n+1} = u_i^n + \frac{\alpha \Delta t}{(\Delta x)^2} \left( u_{i+1}^n - 2u_i^n + u_{i-1}^n \right)
$$


**Ejemplo**

En este ejemplo, se resuelve la ecuación de difusión en un dominio unidimensional con una condición inicial de pulso central. La solución numérica se obtiene utilizando el método de diferencias finitas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Parámetros de la malla
L = 1.0  # Longitud del dominio
T = 0.1  # Tiempo total de simulación
Nx = 50  # Número de puntos en el espacio
Nt = 100  # Número de puntos en el tiempo
alpha = 0.01  # Coeficiente de difusión
dx = L / (Nx - 1)
dt = T / Nt
r = alpha * dt / dx**2
# Inicialización de la malla
x = np.linspace(0, L, Nx)
u = np.zeros(Nx)
u_new = np.zeros(Nx)
# Condiciones iniciales
u[int(Nx/4):int(3*Nx/4)] = 1.0  # Pulso inicial en el centro
# Iteración temporal
for n in range(1, Nt):
	for i in range(1, Nx-1):
    	u_new[i] = u[i] + r * (u[i+1] - 2*u[i] + u[i-1])
	u[:] = u_new[:]
# Visualización del resultado
plt.plot(x, u, label=f'Tiempo t={T}')
plt.xlabel('Posición x')
plt.ylabel('u(x, t)')
plt.title('Solución de la ecuación de difusión')
plt.legend()
plt.show()


## Ejercicio 4

Se desea resolver la ecuación del calor con condiciones de Dirichlet homogéneas:

$$
\left\{\begin{array}{rcll}
u_t(x,t)& = & u_{xx}(x,t)& \textrm{ para } x\in[-1,1],\; t\in[0,T]\\
u(-1,t) &= &u(1,t)=0\, &\forall t
\end{array}\right.
$$

Para ello se propone un esquema centrado en el espacio y explícito en el tiempo:

$$
\frac{U_{j}^{n+1}-U_j^n}{\Delta t} =  \frac{U_{j+1}^n-2U_j^n+U_{j-1}^n}{\Delta x^2}.
$$

- Escribir el esquema de forma matricial: $U^{n+1} = AU^n$ para una matriz $A$ adecuada.
- Resolver numéricamente para $\Delta x=0.05$, $\Delta t=0.0012$ y $\Delta t=0.0013$. ¿Qué se observa?

Testear la resolución para los siguientes datos iniciales:

$$
u_0(x) = \left\{\begin{array}{rl}
					x+1 & \textrm{ si }x<0 \\
					1-x  & \textrm{ si }x\ge 0
					\end{array}\right.,
					\quad\quad
$$

In [ ]:
# # # COMPLETAR

---
# Ecuación de Onda

Consideremos ahora la ecuación de onda unidimensional:

$$
u_{tt} = c^2 u_{xx}
$$

Discretizando en el espacio y el tiempo, obtenemos:

$$
\frac{u_i^{n+1} - 2u_i^n + u_i^{n-1}}{\Delta t^2} = c^2 \frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{\Delta x^2}
$$

Despejando $u_i^{n+1}$:

$$
u_i^{n+1} = 2u_i^n - u_i^{n-1} + \left( \frac{c \Delta t}{\Delta x} \right)^2 \left( u_{i+1}^n - 2u_i^n + u_{i-1}^n \right)
$$


**Ejemplo**

En este código, se resuelve la ecuación de onda en un dominio unidimensional con una condición inicial de pulso central. La solución numérica se obtiene utilizando el método de diferencias finitas.

In [ ]:
# Parámetros de la malla
L = 1.0  # Longitud del dominio
T = 0.1  # Tiempo total de simulación
Nx = 50  # Número de puntos en el espacio
Nt = 100  # Número de puntos en el tiempo
c = 1.0  # Velocidad de la onda
dx = L / (Nx - 1)
dt = T / Nt
r = (c * dt / dx)**2
# Inicialización de la malla
x = np.linspace(0, L, Nx)
u = np.zeros(Nx)
u_new = np.zeros(Nx)
u_old = np.zeros(Nx)
# Condiciones iniciales
u[int(Nx/4):int(3*Nx/4)] = 1.0  # Pulso inicial en el centro
u_old[:] = u[:]
# Iteración temporal
for n in range(1, Nt):
	for i in range(1, Nx-1):
    	u_new[i] = 2*u[i] - u_old[i] + r * (u[i+1] - 2*u[i] + u[i-1])
	u_old[:] = u[:]
	u[:] = u_new[:]
# Visualización del resultado
plt.plot(x, u, label=f'Tiempo t={T}')
plt.xlabel('Posición x')
plt.ylabel('u(x, t)')
plt.title('Solución de la ecuación de onda')
plt.legend()
plt.show()


## Ejercicio 5

Para la ecuación de ondas
$$
U_{tt} = U_{xx} \qquad x\in(0,1), \qquad t>0
$$

considere el método explícito que se obtiene al tomar diferencias centradas en $x$ y en $t$

$$
u_{j}^{n+1}-2u_{j}^{n}+u_{j}^{n-1}=r(u_{j+1}^{n}-2u_{j}^{n}+u_{j-1}^{n}),
$$

donde ahora tomamos $r=(\frac{\Delta t}{\Delta x})^2$

- Implementar el m\'etodo, para las condiciones de contorno $U(0,t)=U(1,t)=0$ e iniciales
$$
U(x,0)=\frac{1}{8} \sin (\pi x) ,\ \ \ U_t(x,0)=0.
$$

    Probarla para distintos valores de $\Delta t$,$\Delta x$.
- Resolver con condiciones iniciales  $U(x,0)=U_t(x,0)=0$ y condiciones de contorno dadas por $U(0,t)=0$, $U(1,t)= {\rm sen}(t)$ y $U(1,t)= {\rm sen}(t)\chi_{[0,\pi]}(t)$.


In [ ]:
# # # COMPLETAR

---
# Consideraciones Adicionales

- **Estabilidad:** La elección de $\Delta t$ y $\Delta x$ es crucial para la estabilidad de los métodos numéricos. Para la ecuación de difusión, se debe cumplir la condición de estabilidad de Courant-Friedrichs-Lewy (CFL):

$$
\alpha \Delta t / (\Delta x)^2 \leq 0.5
$$

Para la ecuación de onda, se debe cumplir

$$
c \Delta t / \Delta x \leq 1
$$

- **Condiciones de contorno:** Es importante aplicar condiciones de contorno adecuadas (Dirichlet, Neumann, periódicas, etc.) para obtener soluciones físicas.

- **Dimensiones superiores:** Los métodos de diferencias finitas se pueden extender a problemas en dos o más dimensiones, aunque la complejidad computacional aumenta significativamente. Estas técnicas y consideraciones permiten resolver una amplia gama de problemas descritos por PDEs en física y matemática aplicada, proporcionando una base sólida para análisis numéricos avanzados.
